In [ ]:
#| default_exp gstreamer.examples

# Gstreamer Examples and tests
> Gstreamer video capture with on/off valve



#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| export
from UAV.imports import *   # TODO why is this relative import on nbdev_export?
from fastcore.utils import *
import cv2
import gi
import numpy as np
from imutils import resize
# from ping_ip import ping_ip
import threading
from multiprocessing import Process
from gi.repository import Gst
import subprocess
import platform

import paho.mqtt.client as mqtt_client

import time
# from dataloader import LoadImages, resize
from pathlib import Path
import logging
# import .nbs.Gstreamer.gst_parameters as params
#| export
gi.require_version('Gst', '1.0')

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev.showdoc import *
from fastcore.test import *


### Default parameters
Overide these default parameters for application specific applications.

The code is shown below:

In [ ]:
#| exports
#|code-fold: true

from dataclasses import dataclass

@dataclass
class DefaultParams():
    camera_dev = "CAM-0"
    cameras = {
        "CAM-0": {
            "gst": [
                'videotestsrc pattern=smpte is-live=true ! tee name=t ',
                't. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=10/1,format=(string)BGR ! ',
                '   videoconvert ! appsink name=sink emit-signals=true  sync=false async=false  max-buffers=2 drop=true ',
                't. ! queue leaky=2 ! valve name=myvalve drop=true ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc ! rtph264pay ! udpsink host=127.0.0.1 port=5000',
                ],
            "udp": True,
            "host": "127.0.0.1",
            "port": 5000,
        },
        "CAM-1": {
            "gst": [
                'videotestsrc pattern=ball is-live=true ! tee name=t ',
                't. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=10/1,format=(string)BGR ! ',
                '   videoconvert ! appsink name=sink emit-signals=true  sync=false async=false  max-buffers=2 drop=true ',
                't. ! queue leaky=2 ! valve name=myvalve drop=true ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc ! rtph264pay ! udpsink host=127.0.0.1 port=5001',
                ],
            "udp": True,
            "host": "127.0.0.1",
            "port": 5001,
        },
        "CAM-2": {
            "gst": [
                'videotestsrc pattern=snow is-live=true ! tee name=t ',
                't. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=10/1,format=(string)BGR ! ',
                '   videoconvert ! appsink name=sink emit-signals=true  sync=false async=false  max-buffers=2 drop=true ',
                't. ! queue leaky=2 ! valve name=myvalve drop=true ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc ! rtph264pay ! udpsink host=127.0.0.1 port=5002',
                ],
            "udp": True,
            "host": "127.0.0.1",
            "port": 5002,
        },
        "CAM-3": {
            "gst": [
                'videotestsrc pattern=pinwheel is-live=true ! tee name=t ',
                't. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=10/1,format=(string)BGR ! ',
                '  videoconvert ! appsink name=sink emit-signals=true  sync=false async=false  max-buffers=2 drop=true ',
                't. ! queue leaky=2 ! valve name=myvalve drop=true ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc ! rtph264pay ! udpsink host=127.0.0.1 port=5003',
                ],
            "udp": True,
            "host": "127.0.0.1",
            "port": 5003,
            },
    
       }

    # socket address and port
    mqqt_address='127.0.0.1'
    src_port=1234

As an example for camera 0, ```DefaultParams.cameras["CAM-0"]["gst"]``` is a list of gstreamer setup commands.

In [ ]:
#|eval: false     don't run this cell in testing as the mqtt broker may not be running
# Test with GstStream

params = DefaultParams()
gstcommand = params.cameras["CAM-0"]["gst"]
with  GstStream("CAM-0", gstcommand) as video:
    with Mqtt("CAM-0", video) as mqtt:
        mqtt.wait_connection()   # wait for connection
        mqtt.client.publish("STREAM-CAMERA", "CAM-0")
        time.sleep(0.1)
        vs = video.get_valve_state("myvalve")
        print(vs)
        test_eq(vs, False)    # ie dont drop frames on this camera, drop on others

        mqtt.client.publish("STREAM-CAMERA", "CAM-1")
        time.sleep(0.1)
        vs = video.get_valve_state("myvalve")
        print(vs)
        test_eq(vs, True)   # ie do drop frames on this camera, don't drop on CAM-1


### Main function for local testing


#### Receive on UDP with this Test  :
from first terminal run 
```sh
   gst-launch-1.0 udpsrc port=5000 ! application/x-rtp,encoding-name=H264,payload=96 ! \
           rtph264depay ! h264parse ! queue ! avdec_h264 ! xvimagesink sync=false async=false -e
  ```
from second terminal run 
```sh
  mosquitto_pub -m "CAM-0" -t "STREAM-CAMERA"
  mosquitto_pub -m "CAM-1" -t "STREAM-CAMERA"
```

### Test with two cameras from terminal in separte processes
The idea is to run the 4 gst pipelines in direent processes

In [ ]:
#|exports
#|eval: false     don't run this cell in testing
#|code-fold: true

from multiprocessing import Process   # you will need to import Process from multiprocessing

if __name__ == '__main__':

    cams = []
    params = DefaultParams()
    for cam in list(params.cameras.keys())[:2]:
        logger.info("Starting Cam: {cam}")
        p = Process(target=main, args=(cam,))
        p.start()
        cams.append(p)

    for p in cams:
        p.join()
        

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()